# Notebook to Detect 3D Structures

In [1]:
import json
import trimesh
import numpy as np

In [2]:
# Meshes for collision checking
def _filter_small_segments(segments, min_length=1e-6):
    """Filter out segments that are too short."""
    valid_segments = []
    for segment in segments:
        start, end = segment
        length = np.linalg.norm(np.array(end) - np.array(start))
        if length >= min_length:
            valid_segments.append(segment)
    return valid_segments


def _generate_mesh(segments, height=2.0, width=0.2, color=None):
    segments = np.array(segments, dtype=np.float64)
    starts, ends = segments[:, 0, :], segments[:, 1, :]
    directions = ends - starts
    lengths = np.linalg.norm(directions, axis=1, keepdims=True)
    unit_directions = directions / lengths
    
    # Create the base box mesh with the height along the z-axis
    base_box = trimesh.creation.box(extents=[1.0, width, height])
    base_box.apply_translation([0.5, 0, 0])  # Align box's origin to its start
    
    # Prepare rotation matrices around the z-axis
    z_axis = np.array([0, 0, 1])  # The desired vertical axis
    angles = np.arctan2(unit_directions[:, 1], unit_directions[:, 0])  # Rotation in the XY plane

    rectangles = []
    lengths = lengths.flatten()

    for i, (start, length, angle) in enumerate(zip(starts, lengths, angles)):
        # Copy the base box and scale to match segment length
        scaled_box = base_box.copy()
        scaled_box.apply_scale([length, 1.0, 1.0])
        
        # Apply rotation around the z-axis
        rotation_matrix = trimesh.transformations.rotation_matrix(angle, z_axis)
        scaled_box.apply_transform(rotation_matrix)
        
        # Translate the box to the segment's starting point
        scaled_box.apply_translation(start)
        if color is not None:
            scaled_box.visual.face_colors = color
        rectangles.append(scaled_box)

    # Concatenate all boxes into a single mesh
    mesh = trimesh.util.concatenate(rectangles)
    return mesh


def _create_agent_box_mesh(position, heading, length, width, height, color=None):
    # Create box centered at origin
    box = trimesh.creation.box(extents=[length, width, height])
    
    # Rotate box to align with heading
    z_axis = np.array([0, 0, 1])
    rotation_matrix = trimesh.transformations.rotation_matrix(heading, z_axis)
    box.apply_transform(rotation_matrix)
    
    # Move box to position
    box.apply_translation(position)
    
    if color is not None:
        box.visual.face_colors = color
    
    return box

In [3]:
def check_3d_structures(scene_path, tolerance=0.1):
    """Check for 3D structures using the same logic as process_waymo_files.py"""
    with open(scene_path, 'r') as f:
        scene = json.load(f)

    # Collect edge points like in the processing script
    edge_points = []
    edge_segments = []
    for road in scene['roads']:
        if road["type"] == "road_edge":
            edge_vertices = [[r["x"], r["y"], r["z"]] for r in road["geometry"]]
            edge_points.extend(edge_vertices)
            edge_segments.extend([
                [edge_vertices[i], edge_vertices[i + 1]]
                for i in range(len(edge_vertices) - 1)
            ])

    # Check for 3D structures using same logic as processing script
    has_3d = False
    if len(edge_points) > 0:
        edge_points = np.array(edge_points)
        xy_points = edge_points[:, :2]
        
        # Process in chunks like in processing script
        chunk_size = 1000
        for i in range(0, len(xy_points), chunk_size):
            chunk = xy_points[i:i + chunk_size]
            # Calculate distances between current chunk and all points
            dists = np.linalg.norm(chunk[:, np.newaxis] - xy_points, axis=2)
            potential_pairs = np.where((dists < tolerance) & (dists > 0))
            
            # Check z-values for identified pairs
            for p1, p2 in zip(*potential_pairs):
                p1_idx = i + p1  # Adjust index for chunking
                if abs(edge_points[p1_idx, 2] - edge_points[p2, 2]) > tolerance:
                    has_3d = True
                    print(f"Found 3D structure between points:")
                    print(f"Point 1: {edge_points[p1_idx]}")
                    print(f"Point 2: {edge_points[p2]}")
                    print(f"Z difference: {abs(edge_points[p1_idx, 2] - edge_points[p2, 2])}")
                    break
            
            if has_3d:
                break

    # Create visualization identical to processing script
    edge_segments = _filter_small_segments(edge_segments)
    edge_mesh = _generate_mesh(edge_segments)

    # Add objects just like in processing script
    object_meshes = []
    trajectory_meshes = []
    for object in scene['objects']:
        if object['type'] not in ['vehicle', 'cyclist', 'pedestrian']:
            continue

        first_valid_idx = next((i for i, valid in enumerate(object["valid"]) if valid), None)
        if first_valid_idx is not None:
            initial_pos = [
                object["position"][first_valid_idx]["x"],
                object["position"][first_valid_idx]["y"],
                object["position"][first_valid_idx]["z"]
            ]
            initial_heading = object["heading"][first_valid_idx]
            color = (180, 0, 0, 15) if object["mark_as_expert"] else (0, 180, 0, 15)
            
            initial_box = _create_agent_box_mesh(
                initial_pos,
                initial_heading,
                object["length"],
                object["width"],
                object["height"],
                color=color
            )
            object_meshes.append(initial_box)
            if object['type'] == 'pedestrian':
                continue
            
            # Create trajectory visualization
            if False in object["valid"]:
                trajectory_segments = []
                for i in range(len(object["position"]) - 1):
                    if object["valid"][i] and object["valid"][i + 1]:
                        trajectory_segments.append([
                            [object["position"][i]["x"], object["position"][i]["y"], object["position"][i]["z"]],
                            [object["position"][i+1]["x"], object["position"][i+1]["y"], object["position"][i+1]["z"]]
                        ])
            else:
                object_vertices = [[pos["x"], pos["y"], pos["z"]] for pos in object["position"]]
                trajectory_segments = [[object_vertices[i], object_vertices[i+1]] for i in range(len(object_vertices) - 1)]
            
            trajectory_segments = _filter_small_segments(trajectory_segments)
            if len(trajectory_segments) > 0:
                color = (180,0,0,15) if object["mark_as_expert"] else (0,180,0,15)
                trajectory_mesh = _generate_mesh(trajectory_segments, color=color)
                trajectory_meshes.append(trajectory_mesh)

    # Create scene
    view = trimesh.Scene()
    view.add_geometry(edge_mesh, node_name="Road Edges")
    for i, mesh in enumerate(trajectory_meshes):
        view.add_geometry(mesh, node_name=f"Vehicle {i} Path")
    for i, mesh in enumerate(object_meshes):
        view.add_geometry(mesh, node_name=f"Vehicle {i} Initial Position")

    print(f"\nHas 3D structures: {has_3d}")
    return view, has_3d

In [5]:
# Example usage:
scene_path = '../data/processed/examples/tfrecord-00000-of-01000_4.json'
view, has_3d = check_3d_structures(scene_path, tolerance=0.2)
view.show()


Has 3D structures: False
